In [ ]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import os
import seaborn as sns
import umap
import warnings
from collections import OrderedDict
from scipy import signal, stats
from mpl_toolkits import mplot3d
from mpl_toolkits.mplot3d import axes3d
import matplotlib.patches as mpatches

plt.rcParams.update({'figure.max_open_warning': 0})
warnings.simplefilter("ignore")

%matplotlib inline
%run /media/turritopsis/katie/grooming/t1-grooming/grooming_functions.ipynb

# for figure styling
sns.set()
sns.set_style('ticks')

In [ ]:
# load data
behavior = 't1_grooming'
prefix = '/media/turritopsis/katie/apviz/classifiers/2021_02_19'
prefix_out = '/media/turritopsis/katie/grooming/summaries/v3-b2'
data = pd.read_parquet(os.path.join(prefix_out, 't1_grooming_subset_curated.parquet'), engine='fastparquet')

In [ ]:
fps = 300.0 # know this for this dataset

# get the joints to analyze
bodyparts = np.array(['L1A', 'L1B', 'L1C', 'L1D', 'L1E', 
                      'L2A', 'L2B', 'L2C', 'L2D', 'L2E', 
                      'L3A', 'L3B', 'L3C', 'L3D', 'L3E', 
                      'R1A', 'R1B', 'R1C', 'R1D', 'R1E', 
                      'R2A', 'R2B', 'R2C', 'R2D', 'R2E',
                      'R3A', 'R3B', 'R3C', 'R3D', 'R3E'])

bodyparts = [x.replace('_error', '') for x in data.columns if '_error' in x]
bodyparts_xyz = [bp + '_' + x for bp in bodyparts for x in ['x', 'y', 'z']]

angle_types = np.array(['_BC', '_flex', '_rot'])
angle_names_all = get_angle_names(data, angle_types, only_t1 = False)
angle_names = get_angle_names(data, angle_types, only_t1 = True)

angle_vars = [v for v in data.columns
              if some_contains(v, ['_flex', '_rot', '_x', '_y', '_z'])
              and not some_contains(v, ['_d1', '_d2', '_freq', '_range'])
              and v[:2] == 'L1']

bout_numbers = np.unique(np.array(data.behavior_bout))
#fly_dict = get_fly_id(data, bout_numbers)
#videos = get_videos(bout_numbers, data)
#dif_flies = np.unique(list(fly_dict.values()))

In [ ]:
# plot all grooming angles across time 
for j in range(len(angle_names)):
    for n in range(len(bout_numbers)):
    
        fig = plt.figure(figsize = (8,4))
        plt.title(angle_names[j] + ' angles during T1 grooming (bout ' + str(int(bout_numbers[n])) + ')', fontsize = 14)
        
        plt.xlabel('Time (Seconds)', fontsize = 14)
        plt.ylabel('Angle (deg)', fontsize = 14)
        colors = plt.get_cmap('tab20c')(np.arange(len(bout_numbers), dtype=int))          
            
        t1_bout_df = data[data.behavior_bout == bout_numbers[n]]
        t1_angle = np.array(t1_bout_df.iloc[0:][angle_names[j]])
        t = np.array(range(len(t1_angle))) / fps
        plt.plot(t, t1_angle, 'k')  
        
        sns.despine()
        plt.show() 

In [ ]:
# plot all grooming angles across time (left and right on same plot) 
angle_names_u = np.unique([x[1:] for x in angle_names])
for j in range(len(angle_names_u)):
    for n in range(len(bout_numbers)):
    
        fig = plt.figure(figsize = (8,4))
        plt.title('L' + angle_names_u[j] + ' and R' + angle_names_u[j] + ' angles during T1 grooming (bout ' + str(int(bout_numbers[n])) + ')', fontsize = 14)
        
        plt.xlabel('Time (Seconds)', fontsize = 14)
        plt.ylabel('Angle (deg)', fontsize = 14)
        colors = plt.get_cmap('tab20c')(np.arange(len(bout_numbers), dtype=int))          
            
        bout= data[data.behavior_bout == bout_numbers[n]]
        left_angle = np.array(bout.iloc[0:]['L' + angle_names_u[j]])
        right_angle = np.array(bout.iloc[0:]['R' + angle_names_u[j]])
        t = np.array(range(len(left_angle))) / fps
        plt.plot(t, left_angle, label = 'L1') 
        plt.plot(t, right_angle, label = 'R1')
        plt.legend(fontsize = 12)
        
        sns.despine()
        plt.show() 

In [ ]:
# plot all joint angles on the same figure for each type (pool all flies and bouts)

for i in range(len(angle_types)):
    
    fig = plt.figure(figsize = (8,4))
    plt.title('Distributions of Leg Joint Angles During T1 Grooming (' + angle_types[i][1:] + ')', y = 1.02, fontsize = 14)
    plt.xlabel('Angle (deg)', fontsize = 14)
    plt.ylabel('Probability Density', fontsize = 14)
    ang_names = [s for s in list(data.columns) if angle_types[i] in s and '1' in s]
    
    for j in range(len(ang_names)):
                        
        t1 = data.iloc[0:][ang_names[j]]
        t1 = t1[np.isfinite(t1)] # ignores nans 
        
        if len(t1) <= 1: 
            continue        
    
        kernel_t1 = stats.gaussian_kde(t1)    
        t1 = np.linspace(np.percentile(t1, 5), np.percentile(t1, 95), 1000)
        height_t1 = kernel_t1.pdf(t1)                                                     
        plt.plot(t1, height_t1, label = ang_names[j])
    
    sns.despine()
    plt.legend(bbox_to_anchor = (1.0, 1.0), fontsize = 13)
    plt.show()

In [ ]:
# distributions of all bout angles (plot for each angle type for each bout)
for i in range(len(angle_types)):
    
    for k in range(len(bout_numbers)):
        
        bout = data[data.behavior_bout == bout_numbers[k]]
        fig = plt.figure(figsize = (8,4))
        plt.title('Distributions of Leg Joint Angles During T1 Grooming (' + angle_types[i][1:] + ', bout ' + str(int(bout_numbers[k])) + ')', y = 1.02, fontsize = 14)
        plt.xlabel('Angle (deg)', fontsize = 14)
        plt.ylabel('Probability Density', fontsize = 14)
        ang_names = [s for s in list(data.columns) if angle_types[i] in s and '1' in s]
    
        for j in range(len(ang_names)):

            t1 = bout.iloc[0:][ang_names[j]]
            t1 = t1[np.isfinite(t1)] # ignores nans 

            if len(t1) <= 1: 
                continue        

            kernel_t1 = stats.gaussian_kde(t1)    
            t1 = np.linspace(np.percentile(t1, 5), np.percentile(t1, 95), 1000)
            height_t1 = kernel_t1.pdf(t1)                                                     
            plt.plot(t1, height_t1, label = ang_names[j])
    
    plt.legend(bbox_to_anchor = (1.0, 1.0), fontsize = 13)
    sns.despine()
    plt.show()

In [ ]:
# distributions of all bout angles (plot for each angle type for each bout)
fly_data, fly_names_sorted = data_per_fly(data)

for k in range(len(fly_names_sorted)):
    
    for i in range(len(angle_types)):
        
        fly = data[data.flyid == fly_names_sorted[k]]
        fig = plt.figure(figsize = (8,4))
        plt.title('Distributions of Leg Joint Angles During T1 Grooming (' + angle_types[i][1:] + ', fly ' + fly_names_sorted[k] + ')', y = 1.02, fontsize = 14)
        plt.xlabel('Angle (deg)', fontsize = 14)
        plt.ylabel('Probability Density', fontsize = 14)
        ang_names = [s for s in list(data.columns) if angle_types[i] in s and '1' in s]
    
        for j in range(len(ang_names)):

            t1 = fly.iloc[0:][ang_names[j]]
            t1 = t1[np.isfinite(t1)] # ignores nans 

            if len(t1) <= 1: 
                continue        

            kernel_t1 = stats.gaussian_kde(t1)    
            t1 = np.linspace(np.percentile(t1, 5), np.percentile(t1, 95), 1000)
            height_t1 = kernel_t1.pdf(t1)                                                     
            plt.plot(t1, height_t1, label = ang_names[j])
    
            sns.despine()
            plt.legend(bbox_to_anchor = (1.0, 1.0), fontsize = 13)
            plt.show()

In [ ]:
# hilbert transform (envelope) across time for all joint angles   
# upper or lower amplitude 
dist = 15
upper = True

for j in range(len(angle_names)):
    for k in range(len(bout_numbers)):          
            
        t1_bout_df = data[data.behavior_bout == bout_numbers[k]]
        t1_x = np.array(t1_bout_df.iloc[0:][angle_names[j]])
        t = np.array(range(len(t1_x))) / fps
        if len(t1_x) < 60:
            continue
            
        fig = plt.figure(figsize = (8,4))
        plt.title(angle_names[j] + ' Angles During T1 Grooming (bout ' + str(int(bout_numbers[k])) + ')', fontsize = 14)
        
        plt.xlabel('Time (Seconds)', fontsize = 14)
        plt.ylabel('Angles (deg)', fontsize = 14)
        colors = plt.get_cmap('tab20c')(np.arange(len(bout_numbers), dtype=int))
        
        plt.plot(t, t1_x, 'k', label = 'signal')  
        indices = signal.find_peaks(upper*t1_x, distance = dist)[0]
        env, peak_first, peak_last = get_envelope(t1_x, dist = dist, upper = upper)
        plt.plot(t[peak_first:peak_last], env[peak_first:peak_last], 'r', label = 'envelope')
        plt.scatter(t[indices], t1_x[indices], c = 'r', marker = 'o', s = 40)
        plt.legend()       
        sns.despine()
        plt.show()

In [ ]:
# quantifying the mean interval between oscillations 
def get_interval_data(data, angle_types, dists, feature = 'line'):
    
    mean_interval_dict = dict()
    stderr_interval_dict = dict()
    mean_freq_dict = dict()
    stderr_freq_dict = dict()
    bout_numbers = np.unique(np.array(data.behavior_bout))
    feature_list = []
    
    for i in range(len(angle_types)):

        dist = dists[i]
        ang_names = [ang for ang in list(data.columns) if angle_types[i] in ang and '1' in ang]
        ang_names = [ang for ang in ang_names if '_range' not in ang]
        stderr = np.zeros((len(bout_numbers), len(ang_names)))
        mean = np.zeros((len(bout_numbers), len(ang_names)))
        stderr_freqs = np.zeros((len(bout_numbers), len(ang_names)))
        mean_freqs = np.zeros((len(bout_numbers), len(ang_names)))

        for j in range(len(ang_names)):
            
            feature_list = []
            
            for k in range(len(bout_numbers)):

                t1_bout_df = data[data.behavior_bout == bout_numbers[k]]
                t1_x = np.array(t1_bout_df.iloc[0:][ang_names[j]])
                t = np.array(range(len(t1_x))) / fps
                mean[k, j], stderr[k, j], intervals = mean_peak_interval(t1_x, fps, dist = dist)
                freq = 1 / intervals # in hz
                mean_freqs[k, j] = np.nanmean(freq) 
                stderr_freqs[k, j] = np.nanstd(freq) / np.sqrt(len(freq))
                feature_list.append(t1_bout_df['line'].iloc[0])

        n = [len(bout_numbers[~np.isnan(stderr[:,x])]) for x in range(len(ang_names))]
        
        mean_interval_dict[angle_types[i]] = mean
        stderr_interval_dict[angle_types[i]] = stderr
        mean_interval_dict[angle_types[i] + ' mean'] = np.nanmean(mean, 0)
        stderr_interval_dict[angle_types[i] + ' stderr'] = stats.sem(stderr, axis = 0, nan_policy = 'omit')
        
        mean_freq_dict[angle_types[i]] = mean_freqs
        stderr_freq_dict[angle_types[i]] = stderr_freqs
        mean_freq_dict[angle_types[i] + ' mean'] = np.nanmean(mean_freqs, 0)
        stderr_freq_dict[angle_types[i] + ' stderr'] = stats.sem(stderr_freqs, axis = 0, nan_policy = 'omit')
        
    return mean_interval_dict, stderr_interval_dict, mean_freq_dict, stderr_freq_dict, feature_list

In [ ]:
# plot mean interval for each joint averaged across all bouts and scatter 
# points around them 
angle_types_names = ['abduction', 'flexion', 'rotation']
sns.set_style('ticks')
scat = True
dists = np.array([20, 20, 20])
mean_interval_dict, stderr_interval_dict, mean_freq_dict, stderr_freq_dict, feature_list = get_interval_data(data, angle_types, dists)
#cmap = plt.get_cmap('Spectral')
# colors = [cmap(i/(n-0.9999)) for i in range(8)] 

for i in range(len(angle_types)):
    
    mean = mean_interval_dict[angle_types[i]]
    mean_int = mean_interval_dict[angle_types[i] + ' mean']
    stderr_int = stderr_interval_dict[angle_types[i] + ' stderr']
    ang_names = [ang for ang in list(data.columns) if angle_types[i] in ang and '1' in ang]
    ang_names = [ang for ang in ang_names if '_range' not in ang]
    
    fig = plt.figure(figsize = (8,4))
    plt.title('Average grooming period of ' + angle_types_names[i] + ' angles', fontsize = 14)
    plt.xlabel('Joint', fontsize = 14)
    plt.ylabel('Period (seconds)', fontsize = 14) 
    ax = plt.gca()
    n = mean.shape[1]
    colors = sns.color_palette('icefire', n)
    # colors = sns.color_palette('cubehelix', n+1) # plt.rcParams['axes.prop_cycle'].by_key()['color']
    for j in range(n):
        k = j + 1
        sc = 0
        if scat:
            sc = 0.5*np.random.rand(mean.shape[0]) - 0.75 
        else: 
            sc = -0.5
        plt.scatter(np.ones(mean.shape[0]) + j + sc, mean[:, j], s = 2, color = colors[j])
        m = mean_int[j]
        x_b = j + 0.5
        x_t = (j + 1)
        ax.axhline(y = m, xmin = j/n + 0.02, xmax = k/n - 0.02, color = colors[j])
        ax.errorbar(j + 0.5, mean_int[j], yerr = stderr_int[j], fmt = 'none', capsize = 5, color = colors[j])
        plt.xlim([0, n])
        plt.ylim([0, 0.25])
    # ax.errorbar(np.arange(0.5, n + 0.5, 1), mean_int, yerr = stderr_int, fmt = 'none', capsize = 5)
    
    plt.xticks(np.arange(0.5, (len(ang_names)), 1), [ang_names[n] for n in range(len(ang_names))])
    sns.despine()
    # plt.savefig(r'/media/turritopsis/katie/grooming/figures/grooming_freq/grooming_int_' + angle_types_names[i] + '.png', bbox_inches = 'tight') 
    plt.show()

In [ ]:
# plot frequency for each joint averaged across all bouts and scatter 
# points around them 
angle_types_names = ['abduction', 'flexion', 'rotation']
sns.set_style('ticks')
scat = True
dists = np.array([15, 15, 15])
mean_interval_dict, stderr_interval_dict, mean_freq_dict, stderr_freq_dict, feature_list = get_interval_data(data, angle_types, dists)
#cmap = plt.get_cmap('Spectral')
# colors = [cmap(i/(n-0.9999)) for i in range(8)] 

for i in range(len(angle_types)):
    
    mean = mean_freq_dict[angle_types[i]]
    mean_freq = mean_freq_dict[angle_types[i] + ' mean']
    stderr_freq = stderr_freq_dict[angle_types[i] + ' stderr']
    ang_names = [ang for ang in list(data.columns) if angle_types[i] in ang and '1' in ang]
    ang_names = [ang for ang in ang_names if '_range' not in ang]
    
    fig = plt.figure(figsize = (8,4))
    plt.title('Average grooming frequency of ' + angle_types_names[i] + ' angles', fontsize = 14)
    plt.xlabel('Joint', fontsize = 14)
    plt.ylabel('Frequency (Hz)', fontsize = 14) 
    ax = plt.gca()
    n = mean.shape[1]
    colors = sns.color_palette('icefire', n)
    # colors = sns.color_palette('cubehelix', n+1) # plt.rcParams['axes.prop_cycle'].by_key()['color']
    for j in range(n):
        k = j + 1
        sc = 0
        if scat:
            sc = 0.5*np.random.rand(mean.shape[0]) - 0.75 
        else: 
            sc = -0.5
        plt.scatter(np.ones(mean.shape[0]) + j + sc, mean[:, j], s = 2, color = colors[j])
        m = mean_freq[j]
        x_b = j + 0.5
        x_t = (j + 1)
        ax.axhline(y = m, xmin = j/n + 0.02, xmax = k/n - 0.02, color = colors[j])
        ax.errorbar(j + 0.5, mean_freq[j], yerr = stderr_freq[j], fmt = 'none', capsize = 5, color = colors[j])
        plt.xlim([0, n])
        plt.ylim([0, 26])
    # ax.errorbar(np.arange(0.5, n + 0.5, 1), mean_int, yerr = stderr_int, fmt = 'none', capsize = 5)
    
    plt.xticks(np.arange(0.5, (len(ang_names)), 1), [ang_names[n] for n in range(len(ang_names))])
    sns.despine()
    # plt.savefig(r'/media/turritopsis/katie/grooming/figures/grooming_freq/grooming_freq_' + angle_types_names[i] + '.png', bbox_inches = 'tight') 
    plt.show()

In [ ]:
# plot frequency for each joint averaged across all bouts and scatter 
# points around them 
angle_types_names = ['abduction', 'flexion', 'rotation']
sns.set_style('ticks')
scat = True
dists = np.array([15, 15, 15])
mean_interval_dict, stderr_interval_dict, mean_freq_dict, stderr_freq_dict, feature_list = get_interval_data(data, angle_types, dists)
#cmap = plt.get_cmap('Spectral')
# colors = [cmap(i/(n-0.9999)) for i in range(8)] 

lines = np.unique(data.line)
colors = sns.color_palette('CMRmap', len(lines))
color_dict = dict(zip(lines, colors))
color_list = []
for feature in feature_list:
    color_list.append(color_dict[feature])
    
patch_list = []
for line in lines:
    patch_list.append(mpatches.Patch(color=color_dict[line], label=line))

for i in range(len(angle_types)):
    
    mean = mean_freq_dict[angle_types[i]]
    mean_freq = mean_freq_dict[angle_types[i] + ' mean']
    stderr_freq = stderr_freq_dict[angle_types[i] + ' stderr']
    ang_names = [ang for ang in list(data.columns) if angle_types[i] in ang and '1' in ang]
    ang_names = [ang for ang in ang_names if '_range' not in ang]
    
    fig = plt.figure(figsize = (8,4))
    plt.title('Average grooming frequency of ' + angle_types_names[i] + ' angles', fontsize = 14)
    plt.xlabel('Joint', fontsize = 14)
    plt.ylabel('Frequency (Hz)', fontsize = 14) 
    ax = plt.gca()
    n = mean.shape[1]
    # colors = sns.color_palette('cubehelix', n+1) # plt.rcParams['axes.prop_cycle'].by_key()['color']
    for j in range(n):
        k = j + 1
        sc = 0
        if scat:
            sc = 0.5*np.random.rand(mean.shape[0]) - 0.75 
        else: 
            sc = -0.5
        plt.scatter(np.ones(mean.shape[0]) + j + sc, mean[:, j], s = 2, c = color_list, alpha = 0.7)
        m = mean_freq[j]
        x_b = j + 0.5
        x_t = (j + 1)
        ax.axhline(y = m, xmin = j/n + 0.02, xmax = k/n - 0.02, color = 'k')
        ax.errorbar(j + 0.5, mean_freq[j], yerr = stderr_freq[j], fmt = 'none', capsize = 5, color = 'k')
        plt.xlim([0, n])
        plt.ylim([0, 26])
    # ax.errorbar(np.arange(0.5, n + 0.5, 1), mean_int, yerr = stderr_int, fmt = 'none', capsize = 5)
    
    plt.xticks(np.arange(0.5, (len(ang_names)), 1), [ang_names[n] for n in range(len(ang_names))])
    sns.despine()
    # plt.savefig(r'/media/turritopsis/katie/grooming/figures/grooming_freq/grooming_freq_' + angle_types_names[i] + '.png', bbox_inches = 'tight') 
    plt.show()
    
fig = plt.figure(figsize = (3,3))
plt.legend(handles = patch_list)
plt.axis('off')
plt.show()

In [ ]:
# plot distributions of mean interval for each joint averaged across all bouts 
#cmap = plt.get_cmap('Spectral')
#colors = [cmap(i/(n-0.9999)) for i in range(8)] 
dists = np.array([25, 25, 25])
mean_interval_dict, stderr_interval_dict, mean_freq_dict, stderr_freq_dict = get_interval_data(data, angle_types, dists)
angle_types_names = ['abduction', 'flexion', 'rotation']

for i in range(len(angle_types)):
    
    mean = mean_interval_dict[angle_types[i]]
    mean_int = mean_interval_dict[angle_types[i]]
    stderr_int = stderr_interval_dict[angle_types[i]]
    ang_names = [ang for ang in list(data.columns) if angle_types[i] in ang and '1' in ang]
    
    fig = plt.figure(figsize = (8,4))
    ax = plt.gca()
    ax.set_title('Distribution of average grooming period of ' + angle_types_names[i] + ' angles', fontsize = 14)
    plt.ylabel('Probability density', fontsize = 14)
    plt.xlabel('Average grooming period (seconds)', fontsize = 14) 
    n = mean.shape[1]
    colors = sns.color_palette('icefire', n)
    # colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

    for j in range(n):
        t1 = mean_int[:, j]
        t1 = t1[np.isfinite(t1)] # ignores nans 

        if len(t1) <= 1: 
            continue        

        kernel_t1 = stats.gaussian_kde(t1)    
        t1 = np.linspace(np.percentile(t1, 5), np.percentile(t1, 95), 1000)
        height_t1 = kernel_t1.pdf(t1)                                                     
        plt.plot(t1, height_t1, label = ang_names[j], color = colors[j])

    plt.legend(bbox_to_anchor = (1.0, 1.0))
    sns.despine()
    # plt.savefig(r'/media/turritopsis/katie/grooming/figures/grooming_freq/grooming_int_dist_' + angle_types_names[i] + '.png', bbox_inches = 'tight')
    plt.show()

In [ ]:
# plot distributions of mean interval for each joint averaged across all bouts 
#cmap = plt.get_cmap('Spectral')
#colors = [cmap(i/(n-0.9999)) for i in range(8)] 
angle_types = np.array(['_abduct', '_flex', '_rot'])
dists = np.array([25, 25, 25])
mean_interval_dict, stderr_interval_dict, mean_freq_dict, stderr_freq_dict = get_interval_data(data, angle_types, dists)
angle_types_names = ['abduction', 'flexion', 'rotation']
colors = sns.color_palette('Blues_r', 5)
rcolors = sns.color_palette('Reds_r', 5)
leg_colors = {'L1A':colors[0] , 'R1A':rcolors[0], 'L1B':colors[1], 'R1B':rcolors[1],
              'L1C':colors[2], 'R1C':rcolors[2], 'L1D':colors[3], 'R1D':rcolors[3]}

for i in range(len(angle_types)):
    
    mean = mean_freq_dict[angle_types[i]]
    mean_freq = mean_freq_dict[angle_types[i]]
    stderr_freq = stderr_freq_dict[angle_types[i]]
    ang_names = [ang for ang in list(data.columns) if angle_types[i] in ang and '1' in ang]
    
    fig = plt.figure(figsize = (8,4))
    ax = plt.gca()
    ax.set_title('Distribution of ' + angle_types_names[i] + ' angle frequencies', fontsize = 14)
    plt.ylabel('Probability density', fontsize = 14)
    plt.xlabel('Average grooming frequency (Hz)', fontsize = 14) 
    n = mean.shape[1]
    colors = sns.color_palette('icefire', n)
    # colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

    for j in range(n):
        t1 = mean_freq[:, j]
        t1 = t1[np.isfinite(t1)] # ignores nans 

        if len(t1) <= 1: 
            continue        

        kernel_t1 = stats.gaussian_kde(t1)    
        t1 = np.linspace(np.percentile(t1, 2), np.percentile(t1, 99), 1000)
        height_t1 = kernel_t1.pdf(t1)                                                     
        plt.plot(t1, height_t1, label = ang_names[j], color = leg_colors[ang_names[j][:3]])
        plt.xlim([5, 12])
        plt.ylim([0, 1])

    # plt.legend(bbox_to_anchor = (1.0, 1.0))
    sns.despine()
    # plt.savefig(r'/media/turritopsis/katie/grooming/figures/grooming_freq/grooming_freq_dist_' + angle_types_names[i] + '.png', bbox_inches = 'tight')
    plt.show()

In [ ]:
# plot distributions of mean interval for each joint averaged across all bouts (for each fly)
#cmap = plt.get_cmap('Spectral')
#colors = [cmap(i/(n-0.9999)) for i in range(8)] 

fly_data, fly_names_sorted = data_per_fly(data)  
dists = np.array([20, 20, 20])

for fly in fly_names_sorted:
    
    fly_data = data[data.flyid == fly]
    mean_interval_dict, stderr_interval_dict = get_interval_data(fly_data, angle_types, dists)
    
    for i in range(len(angle_types)):
        
        mean = mean_interval_dict[angle_types[i]]
        mean_int = mean_interval_dict[angle_types[i]]
        stderr_int = stderr_interval_dict[angle_types[i]]
        ang_names = [ang for ang in list(fly_data.columns) if angle_types[i] in ang and '1' in ang]

        fig = plt.figure(figsize = (8,4))
        ax = plt.gca()
        ax.set_title('Mean Intervals of Grooming Oscillations (' + angle_types[i][1:] + ', fly ' + str(fly) + ')', fontsize = 14)
        plt.ylabel('Probability Density', fontsize = 14)
        plt.xlabel('Mean Grooming Period (seconds)', fontsize = 14) 
        n = mean.shape[1]
        colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

        for j in range(n):
            t1 = mean_int[:, j]
            t1 = t1[np.isfinite(t1)] # ignores nans 

            if len(t1) <= 1: 
                continue        

            kernel_t1 = stats.gaussian_kde(t1)    
            t1 = np.linspace(np.percentile(t1, 5), np.percentile(t1, 95), 1000)
            height_t1 = kernel_t1.pdf(t1)                                                     
            plt.plot(t1, height_t1, label = ang_names[j], color = colors[j])

        plt.legend(bbox_to_anchor = (1.0, 1.0))
        sns.despine()
        plt.show()

In [ ]:
# plot mean interval for each joint for each bout with points scattered 
# around the mean
dist = 30
scat = True
    
for k in range(len(angle_types)):
    
    ang_names = [ang for ang in angle_names if angle_types[k] in ang and '1' in ang]
    stderr = np.zeros((len(bout_numbers), len(ang_names)))
    mean = np.zeros((len(bout_numbers), len(ang_names))) 
    n = mean.shape[1]
    
    for i in range(len(bout_numbers)):
        
        fig = plt.figure(figsize = (8,4))
        plt.title('Mean Intervals of Grooming Oscillations (bout ' + str(int(bout_numbers[i])) +')', fontsize = 14)
        plt.xlabel('Joint', fontsize = 14)
        plt.ylabel('Mean Interval (seconds)', fontsize = 14) 
        ax = plt.gca()
        colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
    
        for j in range(len(ang_names)):
              
            t1_bout_df = data[data.behavior_bout == bout_numbers[i]]
            t1_x = np.array(t1_bout_df.iloc[0:][ang_names[j]])
            mean[i, j], stderr[i, j], intervals = mean_peak_interval(t1_x, fps, dist = dist)
            
            k = j + 1
            sc = 0
            if scat:
                sc = 0.5*np.random.rand(intervals.shape[0]) - 0.75 
            else: 
                sc = -0.5
            plt.scatter(np.ones(intervals.shape[0]) + j + sc, intervals, color = colors[j])
            m = mean[i, j]
            x_b = j + 0.5
            x_t = (j + 1)
            ax.axhline(y = m, xmin = j/n + 0.02, xmax = k/n - 0.02, color = colors[j])
            ax.errorbar(j + 0.5, mean[i, j], yerr = stderr[i, j], fmt = 'none', capsize = 5, color = colors[j])
            
        plt.xlim([0,n])
        plt.ylim([0, 0.3])        
        plt.xticks(np.arange(0.5, len(ang_names), 1), [ang_names[n] for n in range(len(ang_names))])
        sns.despine()
        plt.show()

In [ ]:
# distribution of intervals for each joint for all bouts (not from the mean intervals) 
dists = np.array([25, 30, 30])
    
for k in range(len(angle_types)):
    
    dist = dists[k]
    ang_names = [ang for ang in angle_names if angle_types[k] in ang and '1' in ang] 
    stderr = np.zeros((len(bout_numbers), len(ang_names)))
    mean = np.zeros((len(bout_numbers), len(ang_names))) 
    
    fig = plt.figure(figsize = (8,4))   
    plt.title('Distribution of Grooming Period (' + angle_types[k][1:] +', all bouts)', fontsize = 14)
    plt.xlabel('Grooming Period (seconds)', fontsize = 14)
    plt.ylabel('Probability Density', fontsize = 14) 
    ax = plt.gca()
    colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
            
    for j in range(len(ang_names)):
        
        interval_vec = []
        for i in range(len(bout_numbers)):
              
            bout = data[data.behavior_bout == bout_numbers[i]]
            t1_x = np.array(bout.iloc[0:][ang_names[j]])
            mean[i, j], stderr[i, j], intervals = mean_peak_interval(t1_x, fps, dist = dist)
            interval_vec.extend(intervals)
        
        if len(interval_vec) <= 1:
            continue
        t1 = np.array(interval_vec)
        t1 = t1[np.isfinite(t1)] # ignores nans  

        kernel_t1 = stats.gaussian_kde(t1)    
        t1 = np.linspace(np.percentile(t1, 5), np.percentile(t1, 95), 2000)
        height_t1 = kernel_t1.pdf(t1)       
        plt.plot(t1, height_t1, label = ang_names[j], color = colors[j]) 
        plt.legend(bbox_to_anchor = (1.0, 1.0))

    sns.despine()
    plt.show()

In [ ]:
# distribution of intervals for each joint for all bouts (not from the mean intervals) 
# for each fly
dists = np.array([25, 30, 30])
colors = sns.color_palette()

fly_data, fly_names_sorted = data_per_fly(data)  

for fly in fly_names_sorted:

    fly_data = data[data.flyid == fly]

    for k in range(len(angle_types)):

        dist = dists[k]
        ang_names = [ang for ang in angle_names if angle_types[k] in ang and '1' in ang] 
        stderr = np.zeros((len(bout_numbers), len(ang_names)))
        mean = np.zeros((len(bout_numbers), len(ang_names))) 

        fig = plt.figure(figsize = (8,4))   
        plt.title('Mean Intervals of Grooming Oscillations (' + angle_types[k][1:] +', fly ' + fly + ')', fontsize = 14)
        plt.xlabel('Grooming Period (seconds)', fontsize = 14)
        plt.ylabel('Probability Density', fontsize = 14) 
        ax = plt.gca()
        colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

        for j in range(len(ang_names)):

            interval_vec = []
            for i in range(len(bout_numbers)):

                bout = fly_data[fly_data.behavior_bout == bout_numbers[i]]
                t1_x = np.array(bout.iloc[0:][ang_names[j]])
                mean[i, j], stderr[i, j], intervals = mean_peak_interval(t1_x, fps, dist = dist)
                interval_vec.extend(intervals)

            if len(interval_vec) <= 1:
                continue
            t1 = np.array(interval_vec)
            t1 = t1[np.isfinite(t1)] # ignores nans  

            kernel_t1 = stats.gaussian_kde(t1)    
            t1 = np.linspace(np.percentile(t1, 5), np.percentile(t1, 95), 1000)
            height_t1 = kernel_t1.pdf(t1)       
            plt.plot(t1, height_t1, label = ang_names[j], color = colors[j]) 
            plt.legend(bbox_to_anchor = (1.0, 1.0))

        sns.despine()
        plt.show()

In [ ]:
# plot power spectral density for each bout and find max freq

jc = []
freq_argmax_dict = dict()
plot = False

for i in range(len(angle_types)):
    
    ang_names = [ang for ang in angle_names if angle_types[i] in ang and '1' in ang] 
    
    for j in range(len(ang_names)):
        
        jc.append(ang_names[j])     
        
        for k in range(len(bout_numbers)):
     
            bout = data[data.behavior_bout == bout_numbers[k]]
            t1 = bout.iloc[0:][ang_names[j]]
            t1 = t1[np.isfinite(t1)]
            if len(t1) <= 1: 
                continue

            f, pxx = signal.welch(t1, fs=300, nperseg=128)  
            pxx = pxx - np.mean(pxx)
            bout_name = bout_numbers[k]
            if bout_name not in freq_argmax_dict:
                freq_argmax_dict[bout_name] = []

            value = freq_argmax_dict[bout_name]
            value.append(f[np.argmax(pxx)])
            freq_argmax_dict[bout_name] = value 
            
            if plot:
                fig = plt.figure(figsize = (8,4))
                plt.title('Power Spectrum of ' + ang_names[j] + ' Angles (bout ' + str(int(bout_numbers[k])) + ')', fontsize = 14)           
                plt.xlabel('Frequency (Hz)', fontsize = 14)
                plt.ylabel('Power Spectral Density', fontsize = 14)
                colors = plt.get_cmap('tab20c')(np.arange(len(bout_numbers), dtype=int))                                                                  
                plt.plot(f, pxx, label = 'bout ' + str(int(bout_numbers[k])), c = 'k')
                sns.despine()
                plt.show()         
        
freq_argmax_df = pd.DataFrame(freq_argmax_dict, index = jc)
freq_argmax_df = freq_argmax_df.T
path = '/media/turritopsis/katie/grooming/t1-grooming'
csv_name = os.path.join(path, 'argmax_freqs.csv')
freq_argmax_df.to_csv(csv_name)

In [ ]:
# distribution of grooming frequencies for each joint for all bouts (not from the mean intervals) 
    
for k in range(len(angle_types)):
    
    ang_names = [ang for ang in angle_names if angle_types[k] in ang and '1' in ang] 
    fig = plt.figure(figsize = (8,4))   
    plt.title('Distribution of Grooming Frequencies (' + angle_types[k][1:] +', all bouts)', fontsize = 14)
    plt.xlabel('Grooming Frequency (Hz)', fontsize = 14)
    plt.ylabel('Probability Density', fontsize = 14) 
    colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
            
    for j in range(len(ang_names)):
        
        freqs = np.array(freq_argmax_df[ang_names[j]])
        freqs = freqs[np.isfinite(freqs)] # ignores nans     
        
        if len(freqs) <= 1:
            continue  

        kernel = stats.gaussian_kde(freqs)    
        freqs = np.linspace(np.percentile(freqs, 5), np.percentile(freqs, 95), 1000)
        height = kernel.pdf(freqs)       
        plt.plot(freqs, height, label = ang_names[j], color = colors[j]) 
        plt.legend(bbox_to_anchor = (1.0, 1.0))

    sns.despine()
    plt.show()

In [ ]:
# plot average frequency for each joint averaged across all bouts and scatter 
# points around them 
scat = True
#cmap = plt.get_cmap('Spectral')
#colors = [cmap(i/(n-0.9999)) for i in range(8)] 

for i in range(len(angle_types)):
    
    ang_names = [ang for ang in list(data.columns) if angle_types[i] in ang and '1' in ang]
    freqs = freq_argmax_df[ang_names]
    means = np.nanmean(freqs, axis = 0)
    stderr = stats.sem(freqs, axis = 0, nan_policy = 'omit')
    
    fig = plt.figure(figsize = (8,4))
    plt.title('Mean Frequency of Grooming (' + angle_types[i][1:] + ', all bouts)', fontsize = 14)
    plt.xlabel('Joint', fontsize = 14)
    plt.ylabel('Mean Frequency (seconds)', fontsize = 14) 
    ax = plt.gca()
    
    n = len(means)
    colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
    for j in range(n):
        k = j + 1
        sc = 0
        if scat:
            sc = 0.5*np.random.rand(freqs.shape[0]) - 0.75 
        else: 
            sc = -0.5
        plt.scatter(np.ones(freqs.shape[0]) + j + sc, np.array(freqs.iloc[:,j]), s = 2, color = colors[j])
        ax.axhline(y = means[j], xmin = j/n + 0.02, xmax = k/n - 0.02, color = colors[j])
        ax.errorbar(j + 0.5, means[j], yerr = stderr[j], fmt = 'none', capsize = 5, color = colors[j])
        plt.xlim([0,n])
        plt.ylim([0, 16])
    
    plt.xticks(np.arange(0.5, (len(ang_names)), 1), [ang_names[n] for n in range(len(ang_names))])
    sns.despine()
    plt.show()

In [ ]:
np.array(freqs.iloc[:,j])

In [ ]:
# plot envelopes from all bouts all ontop of eachother for each joint and angle 
       
sns.set()
sns.set_style('ticks')
filter_freqs = True
upper = True
dists = np.array([25, 30, 30])
order = 3

for j in range(len(angle_names)): 

    dist = dists[j]
    fig = plt.figure(figsize = (8,4))
    plt.title('Envelopes of ' + angle_names[j] + ' Angles During T1 Grooming (all bouts)', fontsize = 14)
    plt.xlabel('Time (Seconds)', fontsize = 14)
    plt.ylabel('Angle (deg)', fontsize = 14)
    colors = plt.get_cmap('tab20c')(np.arange(len(bout_numbers), dtype=int)) 
        
    for k in range(len(bout_numbers)):                         

        t1_bout_df = data[data.behavior_bout == bout_numbers[k]]
        t1_x = np.array(t1_bout_df.iloc[0:][angle_names[j]])
        t = np.array(range(len(t1_x))) / fps
        
        if filter_freqs: 
            b, a = signal.butter(order, [5, 12], btype='bandpass', fs = fps)
            t1_x = signal.lfilter(b, a, t1_x)
        
        plt.plot(t, t1_x, 'k', label = 'signal')                 
        env, indices = get_envelope_alt(t1_x, dist = dist, upper = upper)
        if len(indices > 5): 
            peak_first = indices[0]
            peak_last = indices[-1]
            plt.plot(t[peak_first:peak_last], env[peak_first:peak_last], 'r', label = 'envelope')
        else:
            continue
        
    # plt.legend()       
    sns.despine()
    plt.show()

In [ ]:
sc